 # Structure Factor GPU Testing & Debugging



 This notebook tests and debugs the GPU-accelerated structure factor calculator.

# Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import sys
import os

# Add parent directory to path so we can import our module
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

# Import our classes
from structure_factor_torch import StructureFactorCalculator, StructureFactorVisualizer

# Check GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")


ImportError: cannot import name 'StructureFactorCalculator' from 'structure_factor_torch' (/Users/gcardoso/Personal/portfolio/PhD/Analysis/structure_factor_torch.py)

# Create Test Data

In [ ]:
# Create simple test datasets
def create_test_data():
    """Create simple test particle arrangements."""
    
    # Create data directory if it doesn't exist
    os.makedirs('data', exist_ok=True)
    os.makedirs('results', exist_ok=True)
    
    # Test 1: Small random arrangement (100 particles)
    np.random.seed(42)
    particles_100 = np.random.rand(100, 2) * 10  # 10x10 area
    np.savetxt('data/test_particles_100.csv', particles_100, delimiter=',')
    print(f"Created test_particles_100.csv: {particles_100.shape}")
    
    # Test 2: Medium random arrangement (1000 particles)
    particles_1000 = np.random.rand(1000, 2) * 20  # 20x20 area
    np.savetxt('data/test_particles_1000.csv', particles_1000, delimiter=',')
    print(f"Created test_particles_1000.csv: {particles_1000.shape}")
    
    # Test 3: Simple grid for validation
    x = np.linspace(0, 5, 6)
    y = np.linspace(0, 5, 6)
    xx, yy = np.meshgrid(x, y)
    grid_particles = np.column_stack([xx.ravel(), yy.ravel()])
    np.savetxt('data/test_grid_6x6.csv', grid_particles, delimiter=',')
    print(f"Created test_grid_6x6.csv: {grid_particles.shape}")
    
    return particles_100, particles_1000, grid_particles

# Create test data
particles_100, particles_1000, grid_particles = create_test_data()

# Visualize test data
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].scatter(particles_100[:, 0], particles_100[:, 1], s=20, alpha=0.7)
axes[0].set_title('Random 100 particles')
axes[0].set_aspect('equal')

axes[1].scatter(particles_1000[:, 0], particles_1000[:, 1], s=5, alpha=0.5)
axes[1].set_title('Random 1000 particles')
axes[1].set_aspect('equal')

axes[2].scatter(grid_particles[:, 0], grid_particles[:, 1], s=50)
axes[2].set_title('6x6 Grid')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()


# Test Calculator Class

In [ ]:
# Initialize calculator
calculator = StructureFactorCalculator()

# Test with small dataset first
print("Testing with 100 particles...")
try:
    calculator.calculate_structure_factor(
        read_folder='data/',
        file='test_particles_100',
        range_calculation=5.0,    # Small range for quick testing
        vector_step=0.5,          # Coarse resolution for speed
        save_folder='results/'
    )
    print("✅ Small test completed successfully!")
except Exception as e:
    print(f"❌ Error in small test: {e}")
    import traceback
    traceback.print_exc()


# Test Parameter Extraction

In [ ]:
# Test parameter calculation function
test_structure = particles_100

D, N = calculator.get_calculation_parameters(test_structure)
print(f"Structure parameters:")
print(f"  Number of particles: {N}")
print(f"  Average neighbor distance: {D:.4f}")
print(f"  Structure bounds: x=[{test_structure[:,0].min():.2f}, {test_structure[:,0].max():.2f}], y=[{test_structure[:,1].min():.2f}, {test_structure[:,1].max():.2f}]")


# Test Vector Generation

In [ ]:
# Test vector generation
import torch

structure_tensor = torch.from_numpy(test_structure)
d_x, d_y, q = calculator.generate_vectors(structure_tensor, 5.0, 0.5, D)

print(f"Distance arrays shape: d_x {d_x.shape}, d_y {d_y.shape}")
print(f"Q-vector shape: {q.shape}")
print(f"Q-vector range: [{q[0]:.3f}, {q[-1]:.3f}] (normalized)")
print(f"Q-vector range: [{q[0]*D:.3f}, {q[-1]*D:.3f}] (physical)")


# GPU Memory Check

In [ ]:
def check_gpu_memory_usage(n_particles, q_points):
    """Estimate GPU memory usage for given parameters."""
    
    # Distance arrays: 2 * (1, N²) * 4 bytes (float32)
    distance_memory = 2 * n_particles**2 * 4
    
    # Q-vector: q_points * 4 bytes
    q_memory = q_points * 4
    
    # Structure factor result: (q_points, q_points) * 8 bytes (complex64)
    result_memory = q_points**2 * 8
    
    total_memory = distance_memory + q_memory + result_memory
    
    print(f"Memory estimate for {n_particles} particles, {q_points} q-points:")
    print(f"  Distance arrays: {distance_memory/1e6:.1f} MB")
    print(f"  Q-vector: {q_memory/1e3:.1f} KB")
    print(f"  Result array: {result_memory/1e6:.1f} MB")
    print(f"  Total: {total_memory/1e6:.1f} MB")
    
    if torch.cuda.is_available():
        available_memory = torch.cuda.get_device_properties(0).total_memory
        print(f"  Available GPU memory: {available_memory/1e9:.1f} GB")
        print(f"  Memory usage: {total_memory/available_memory*100:.1f}%")
    
    return total_memory

# Check memory for different sizes
check_gpu_memory_usage(100, 10)   # Small test
print()
check_gpu_memory_usage(1000, 20)  # Medium test
print()
check_gpu_memory_usage(5000, 50)  # Large test


# Test Visualizer Class

In [ ]:
# Initialize visualizer
visualizer = StructureFactorVisualizer()

# Check if we have results to visualize
import glob
result_files = glob.glob('results/*.dat')
print(f"Found result files: {result_files}")

if result_files:
    # Find a 2D structure factor file
    sq_2d_files = [f for f in result_files if '2D_Sq_' in f]
    if sq_2d_files:
        # Extract filename without path and extension
        file_path = sq_2d_files[0]
        file_name = os.path.basename(file_path).replace('.dat', '')
        
        print(f"Visualizing: {file_name}")
        
        try:
            visualizer.plot_Sq_2D(
                folder_read='results/',
                file=file_name,
                edge=(-2, 2),           # Plot range
                x_axis='qD',            # Normalized coordinates
                save_plot=False,        # Don't save during testing
                log_scale=False,        # Linear scale first
                max_value=5             # Reasonable max for visualization
            )
        except Exception as e:
            print(f"❌ Visualization error: {e}")
            import traceback
            traceback.print_exc()
else:
    print("No result files found. Run the calculator first.")


# Performance Comparison (CPU vs GPU)

In [ ]:
import time

def benchmark_calculation(n_particles, q_points, use_gpu=True):
    """Simple benchmark of calculation speed."""
    
    # Create test data
    test_data = np.random.rand(n_particles, 2) * 10
    np.savetxt('data/benchmark_test.csv', test_data, delimiter=',')
    
    # Calculate parameters
    D, N = calculator.get_calculation_parameters(test_data)
    
    # Generate vectors
    structure_tensor = torch.from_numpy(test_data)
    d_x, d_y, q = calculator.generate_vectors(structure_tensor, 5.0, 5.0/q_points, D)
    
    # Move to appropriate device
    device = 'cuda' if (use_gpu and torch.cuda.is_available()) else 'cpu'
    d_x = d_x.to(device)
    d_y = d_y.to(device)
    q = q.to(device)
    N_tensor = torch.tensor(N, dtype=torch.float).to(device)
    
    print(f"Benchmarking {n_particles} particles, {len(q)} q-points on {device}")
    
    # Time the calculation (just a few q-points for speed)
    start_time = time.time()
    
    # Calculate just a small portion for timing
    n_test = min(3, len(q))  # Test first 3x3 q-points
    structure_factor = torch.zeros((n_test, n_test), device=device)
    
    for a in range(n_test):
        for b in range(n_test):
            structure_factor[a,b] = (1/N_tensor)*torch.sum(torch.exp(1j*(q[a]*d_x + q[b]*d_y)))
    
    end_time = time.time()
    calculation_time = end_time - start_time
    
    # Extrapolate to full calculation
    full_time_estimate = calculation_time * (len(q)/n_test)**2
    
    print(f"  Partial calculation time: {calculation_time:.3f} seconds")
    print(f"  Full calculation estimate: {full_time_estimate:.1f} seconds")
    
    return calculation_time, full_time_estimate

# Run benchmarks
print("Performance Benchmarking:")
print("=" * 50)

# Small test
cpu_time, cpu_full = benchmark_calculation(100, 10, use_gpu=False)
if torch.cuda.is_available():
    gpu_time, gpu_full = benchmark_calculation(100, 10, use_gpu=True)
    speedup = cpu_time / gpu_time
    print(f"  Speedup: {speedup:.1f}x")
else:
    print("  GPU not available for comparison")


# Debug Common Issues

In [ ]:
def debug_common_issues():
    """Check for common issues and provide diagnostics."""
    
    print("Diagnostic Checks:")
    print("=" * 30)
    
    # Check 1: File paths
    print("1. File path check:")
    if os.path.exists('data/'):
        csv_files = glob.glob('data/*.csv')
        print(f"   ✅ Data folder exists with {len(csv_files)} CSV files")
    else:
        print("   ❌ Data folder missing")
    
    if os.path.exists('results/'):
        result_files = glob.glob('results/*.dat')
        print(f"   ✅ Results folder exists with {len(result_files)} result files")
    else:
        print("   ❌ Results folder missing")
    
    # Check 2: GPU availability
    print("\n2. GPU check:")
    if torch.cuda.is_available():
        print(f"   ✅ CUDA available: {torch.cuda.get_device_name()}")
        print(f"   ✅ GPU memory: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")
    else:
        print("   ⚠️  CUDA not available, using CPU")
    
    # Check 3: Memory usage
    print("\n3. Memory check:")
    import psutil
    ram_gb = psutil.virtual_memory().total / 1e9
    print(f"   ✅ System RAM: {ram_gb:.1f} GB")
    
    # Check 4: PyTorch installation
    print("\n4. PyTorch installation:")
    print(f"   ✅ PyTorch version: {torch.__version__}")
    
    # Check 5: Data format
    print("\n5. Data format check:")
    try:
        test_data = np.genfromtxt('data/test_particles_100.csv', delimiter=',')
        print(f"   ✅ Data loads correctly: shape {test_data.shape}")
        if test_data.shape[1] != 2:
            print("   ❌ Data should have exactly 2 columns (x, y)")
        if np.any(np.isnan(test_data)):
            print("   ❌ Data contains NaN values")
    except Exception as e:
        print(f"   ❌ Data loading error: {e}")

# Run diagnostics
debug_common_issues()


# Quick Test Suite

In [ ]:
def run_quick_tests():
    """Run a series of quick tests to verify everything works."""
    
    print("Quick Test Suite:")
    print("=" * 40)
    
    tests_passed = 0
    total_tests = 0
    
    # Test 1: Calculator initialization
    total_tests += 1
    try:
        calc = StructureFactorCalculator()
        print("✅ Test 1: Calculator initialization")
        tests_passed += 1
    except Exception as e:
        print(f"❌ Test 1: Calculator initialization failed - {e}")
    
    # Test 2: Parameter calculation
    total_tests += 1
    try:
        test_data = np.random.rand(50, 2)
        D, N = calc.get_calculation_parameters(test_data)
        assert N == 50, f"Expected N=50, got N={N}"
        assert D > 0, f"Expected D>0, got D={D}"
        print("✅ Test 2: Parameter calculation")
        tests_passed += 1
    except Exception as e:
        print(f"❌ Test 2: Parameter calculation failed - {e}")
    
    # Test 3: Vector generation
    total_tests += 1
    try:
        structure_tensor = torch.from_numpy(test_data)
        d_x, d_y, q = calc.generate_vectors(structure_tensor, 2.0, 0.5, D)
        assert d_x.shape[1] == 50**2, f"Wrong distance array shape"
        assert len(q) > 0, "Q-vector is empty"
        print("✅ Test 3: Vector generation")
        tests_passed += 1
    except Exception as e:
        print(f"❌ Test 3: Vector generation failed - {e}")
    
    # Test 4: Visualizer initialization
    total_tests += 1
    try:
        viz = StructureFactorVisualizer()
        print("✅ Test 4: Visualizer initialization")
        tests_passed += 1
    except Exception as e:
        print(f"❌ Test 4: Visualizer initialization failed - {e}")
    
    print(f"\nTest Results: {tests_passed}/{total_tests} passed")
    
    if tests_passed == total_tests:
        print("🎉 All tests passed! Your code is ready to use.")
    else:
        print("⚠️  Some tests failed. Check the errors above.")

# Run the test suite
run_quick_tests()


# Next Steps

In [ ]:
print("Next Steps for Development:")
print("=" * 35)
print()
print("1. 🧪 Run full calculation with medium dataset:")
print("   calculator.calculate_structure_factor('data/', 'test_particles_1000', 10.0, 0.2, 'results/')")
print()
print("2. 📊 Compare GPU vs CPU performance:")
print("   - Time both calculations")
print("   - Measure memory usage")
print("   - Document speedup achieved")
print()
print("3. 🎨 Test different visualization options:")
print("   - Try log_scale=True for wide dynamic range")
print("   - Test different coordinate systems")
print("   - Save high-quality figures")
print()
print("4. 🔧 Optimize parameters:")
print("   - Find optimal q-range for your structures")
print("   - Balance resolution vs computation time")
print("   - Test with real experimental data")
print()
print("5. 📝 Document performance improvements:")
print("   - Quantify GPU speedup")
print("   - Compare with CPU-only version")
print("   - Create benchmark plots")
